# <strong> Example substance flows and material criticality
   

Example notebook to obtain the battery manufacturing substance flows and calculate material criticality. 

In [3]:
#Import required packages

import pandas as pd
import sys
import matplotlib.pyplot as plt
from tabulate import tabulate


#Import the battery design module from a local path
local_path =r"C:\Users\Joris\OneDrive - Newcastle University\Python\Projects\Batt_Sust_Model\batt_sust_model" #ADD PATH
sys.path.insert(1, local_path)
from battery_design import battery_design as bd
import battery_emissions as bat_lca


#Local path to BatPaC version 5
path_batpac = r"C:\Users\Joris\OneDrive - Newcastle University\PhD\Models\battery_sustainability_model\Case study\BatPaC 5.0 10Feb2022.xlsm"#ADD PATH


## Solve battery design
The design of the Volkswagen MEB platform, 77 kWh version as reported by
[PushEVS.com](https://pushevs.com/2020/05/13/volkswagen-meb-details/) is used as an example (NMC622 is used instead of NMC712, as the latter is not in the public version of BatPaC V5l). All other parameters are BatPaC default parameters.

* usable battery energy: 94%
* Total pack energy: 82 kWh
* Electrode pair: NMC622-G
* Silicon additive anode: 0%
* Cells per module: 24
* Modules per pack: 12
* Modules in parallel: 2 
* Maximum charging time: 33 minutes (ev-database.org)
* Maximum charge power: 135 kW (ev-database.org)
<br>

<!-- <a href=https://www.volkswagen-newsroom.com/en/press-releases/long-range-and-rapid-charging-the-battery-system-is-at-the-heart-of-the-volkswagen-id3-id4-and-id4-gtx-7130>
<center><img src='../docs/ID pack.jpg' width=600/> <br></center>
<figcaption align = "center"><b>Structure of the Volkswagen MEB platform (VW, 2021) </b></figcaption>
</a> -->

In [2]:
#Establish battery system class of specific design:
battery = bd.Battery_system ( vehicle_type='EV', 
                             electrode_pair='NMC622-G (Energy)', 
                             cells_per_module=24, 
                             modules_per_row=6,
                             rows_of_modules=2, 
                             cells_in_parallel=1,
                             silicon_anode= 0.00, 
                             pack_energy=82, 
                             calculate_fast_charge = 'Yes',
                             max_charging_time = 33,  
                             max_charge_power = 135, 
                             available_energy=94)
#Establish parameter dictionary
parameter_dict = battery.parameter_dictionary()

# #Solve battery design in BatPaC returning nested dictionary with material contents and parameter values
# #Make sure BatPaC is not open or running in the background. 
# #BatPaC can run in the foreground if visible=True
result_dict = bd.solve_batpac_battery_system (batpac_path = path_batpac, parameter_dict=parameter_dict, visible=False)

## Calculate material and energy flow foreground system

In [ ]:
# process_formula = r"C:\Users\Joris\OneDrive - Newcastle University\PhD\Models\ME optimisation model\ME_battery_supply_chain\Model\data\process_formulas.xlsx"
# process_formula =pd.read_excel(process_formula, sheet_name = 'activity_functions_cut_off', index_col=[1,2]).T.to_dict()
# project_param_formulas =r"C:\Users\Joris\OneDrive - Newcastle University\PhD\Models\ME optimisation model\ME_battery_supply_chain\Model\data\bw_default_project_parameters.xlsx"
# project_param_formulas = pd.read_excel(project_param_formulas, index_col=0).fillna(0).T.to_dict()


IndexTable

#Empty np array of size total design*py scenarios * electricity_scenario* goods * processes
A_matrix_nested = np.zeros((len(D), IndexTable.loc['Yield_scenario', 'IndexSize'], len(g), len(p)))

#Dictionary of all system parameters:
param_dict_all = {}
A_base = df_A_base.copy(deep=True)

for x in tqdm(D):#pack designs
    for s in range(IndexTable.loc['Yield_scenario', 'IndexSize']):
        process_parameters = {param: value for param, value in zip (IndexTable.loc['Process_parameter', 'Classification'].Items, 
                                                                    ParameterDict['4_PY_battery_manufacturing'].Values[:,s])}
        process_parameters.update({param: value for param, value in zip (IndexTable.loc['Process_parameter', 'Classification'].Items, 
                                                                    ParameterDict['4_PAR_energy_consumption'].Values[:,1]) if value > 0})
        param_dict_all[x] = bat_lca.parameter_dictionary (mc_battery_design_dct[x], process_parameters, bat_design_param_dct[x], project_param_formulas)
        A_matrix = bat_lca.calculate_modular_A(A_base,param_dict_all[x], process_formula, project_param_formulas)
        
        
        A_matrix_nested[x,s,:] = A_matrix

